In [77]:
import numpy as np
from PIL import Image, ImageEnhance
from plantcv import plantcv as pcv
from plantcv.parallel import WorkflowInputs
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import experimental, Conv2D, MaxPooling2D, Dense, Flatten
import os
import argparse



In [2]:
def loadDataset(path, img_size, batch_size):
    return tf.keras.preprocessing.image_dataset_from_directory(
        path, 
        shuffle = True, 
        image_size = (img_size, img_size), 
        batch_size = batch_size
    )

In [3]:
def remove_back(img, size_fill, enhance_val, buffer_size):
	img_img = Image.fromarray(img, mode="RGB")
	contr_img = ImageEnhance.Contrast(img_img).enhance(enhance_val)
	gray_img = pcv.rgb2gray_lab(rgb_img=np.array(contr_img), channel='a')
	thresh = pcv.threshold.triangle(gray_img=gray_img, object_type="dark", xstep= 100)
	edge_ok = pcv.fill(bin_img=thresh, size=5000)
	mask = pcv.fill(bin_img=pcv.invert(gray_img=edge_ok), size=size_fill)
	contours, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
	mask_with_buffer = mask.copy()
	if (len(contours)):
		cv2.drawContours(mask_with_buffer, contours[np.argmax([len(c) for c in contours])], -1, (0, 0, 0), buffer_size)
	if (mask_with_buffer[0,0] == 0 and mask_with_buffer[0,-1] == 0 and mask_with_buffer[0,-1] == 0 and mask_with_buffer[-1,0] == 0):
		mask_with_buffer[0:11,0:11] = 255
		mask_with_buffer[-11:,-11:] = 255
		mask_with_buffer[0:11,-11:] = 255
		mask_with_buffer[-11:,0:11] = 255
	mask_with_buffer[0:1,:] = 255
	mask_with_buffer[-1:,:] = 255
	mask_with_buffer[:,0:1] = 255
	mask_with_buffer[:,-1:] = 255
	mask_with_buffer = pcv.fill(bin_img=mask_with_buffer, size=size_fill)
	img_modified = np.ones_like(img) * 255
	img_modified[mask_with_buffer == 0] = img[mask_with_buffer == 0]
	return img_modified

In [4]:
def get_dataset_partition_tf(ds, train_split = 0.85, shuffle = True, shuffle_size = 10000):
	if shuffle:
		ds = ds.shuffle(shuffle_size, seed = 12)
	len_train_dataset = int(len(ds) * train_split)
	train_dataset = ds.take(len_train_dataset)
	cv_dataset = ds.skip(len_train_dataset)
	return train_dataset, cv_dataset

In [6]:
batch_size = 32
epochs = 15
path = "../leaves/images/"
save_dir = ""
save_name = "learnings"
img_size = 256
input_shape = (batch_size, img_size, img_size, 3)

In [10]:
fn = os.listdir(path)

In [11]:
fn

['Grape_Esca',
 'Apple_rust',
 'Grape_Black_rot',
 'Grape_healthy',
 'Apple_healthy',
 'Grape_spot',
 'Apple_scab',
 'Apple_Black_rot']

str(foldername + "/" + fn)

In [136]:
img_path_list = [[[foldername, fn, '/'.join([elem for elem in foldername.split("/") if elem != ".." and elem != "."])] for fn in filenames] for foldername, subdirectory, filenames in os.walk(path) if len(filenames)]

In [138]:
img_path_list = np.array([element for sous_liste in img_path_list for element in sous_liste])

In [140]:
img_array = np.array([np.array(Image.open(str(img_path[0] + "/" + img_path[1]), "r")) for img_path in img_path_list])

In [143]:
img_back_removed = [remove_back(img, 5000, 1, 10) for img in img_array]

In [ ]:
img_back_removed_IMG = [Image.fromarray(img_array) for img_array in img_back_removed]

In [160]:
makedir = [os.makedirs("result/" + path[2], exist_ok=True) for path in img_path_list]
save = [img.save("result/" + path[2] + "/" + path[1]) for path, img in zip(img_path_list, img_back_removed_IMG)]